# Lookout Smishing & Phishing Threat Hunting

This notebook provides threat hunting queries for investigating SMS phishing (smishing) attacks detected by Lookout.

## Prerequisites
- Lookout data connector configured in Microsoft Sentinel
- LookoutEvents parser deployed
- SMISHING_ALERT events ingested

## 1. Smishing Campaign Detection

Identify potential phishing campaigns by analyzing URLs that target multiple users.

In [ ]:
LookoutEvents
| where EventType == "SMISHING_ALERT"
| where isnotempty(SmishingAlertURL)
| summarize 
    TargetCount = dcount(DeviceEmailAddress),
    FirstSeen = min(TimeGenerated),
    LastSeen = max(TimeGenerated),
    Categories = make_set(SmishingAlertCategory),
    Severities = make_set(SmishingAlertSeverity)
    by SmishingAlertURL
| where TargetCount >= 2
| sort by TargetCount desc

## 2. High-Risk Smishing Alerts

Focus on critical and high severity smishing attempts.

In [ ]:
LookoutEvents
| where EventType == "SMISHING_ALERT"
| where SmishingAlertSeverity in ("CRITICAL", "HIGH")
| project 
    TimeGenerated,
    DeviceEmailAddress,
    DevicePlatform,
    SmishingAlertType,
    SmishingAlertCategory,
    SmishingAlertSeverity,
    SmishingAlertURL,
    SmishingAlertDescription
| sort by TimeGenerated desc

## 3. Smishing by Category Analysis

Understand the types of smishing attacks targeting your organization.

In [ ]:
LookoutEvents
| where EventType == "SMISHING_ALERT"
| summarize 
    Count = count(),
    UniqueTargets = dcount(DeviceEmailAddress)
    by SmishingAlertCategory, SmishingAlertType
| sort by Count desc

## 4. Users Targeted by Multiple Smishing Attempts

Identify users who may be specifically targeted.

In [ ]:
LookoutEvents
| where EventType == "SMISHING_ALERT"
| summarize 
    AttackCount = count(),
    UniqueURLs = dcount(SmishingAlertURL),
    Categories = make_set(SmishingAlertCategory),
    FirstSeen = min(TimeGenerated),
    LastSeen = max(TimeGenerated)
    by DeviceEmailAddress
| where AttackCount >= 3
| sort by AttackCount desc

## 5. Smishing Trend Analysis

Track smishing attack trends over time.

In [ ]:
LookoutEvents
| where EventType == "SMISHING_ALERT"
| summarize Count = count() by bin(TimeGenerated, 1d), SmishingAlertCategory
| render timechart